In [ ]:
!pip install langchain-graph-retriever

In [ ]:
!pip install graph_rag_example_helpers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.5/300.5 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.2/129.2 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 19.6 MB/s eta 0:00:00


In [ ]:
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 27.3 MB/s eta 0:00:00


In [ ]:
from graph_rag_example_helpers.datasets.animals import fetch_documents

In [ ]:
animals=fetch_documents()

In [ ]:
animals

[Document(id='aardvark', metadata={'type': 'mammal', 'number_of_legs': 4, 'keywords': ['burrowing', 'nocturnal', 'ants', 'savanna'], 'habitat': 'savanna', 'tags': [{'a': 5, 'b': 7}, {'a': 8, 'b': 10}]}, page_content='the aardvark is a nocturnal mammal known for its burrowing habits and long snout used to sniff out ants.'),
 Document(id='albatross', metadata={'type': 'bird', 'number_of_legs': 2, 'keywords': ['seabird', 'wingspan', 'ocean'], 'habitat': 'marine', 'tags': [{'a': 5, 'b': 8}, {'a': 8, 'b': 10}]}, page_content='the albatross is a large seabird with the longest wingspan of any bird, allowing it to glide effortlessly over oceans.'),
 Document(id='alligator', metadata={'type': 'reptile', 'number_of_legs': 4, 'keywords': ['reptile', 'jaws', 'wetlands'], 'diet': 'carnivorous', 'nested': {'a': 5}}, page_content='alligators are large reptiles with powerful jaws and are commonly found in freshwater wetlands.'),
 Document(id='alpaca', metadata={'type': 'mammal', 'number_of_legs': 4, '

In [ ]:
len(animals)

99

In [ ]:
animals[0].page_content

'the aardvark is a nocturnal mammal known for its burrowing habits and long snout used to sniff out ants.'

In [ ]:
animals[0].metadata

{'type': 'mammal',
 'number_of_legs': 4,
 'keywords': ['burrowing', 'nocturnal', 'ants', 'savanna'],
 'habitat': 'savanna',
 'tags': [{'a': 5, 'b': 7}, {'a': 8, 'b': 10}]}

In [ ]:
animals[1].page_content

'the albatross is a large seabird with the longest wingspan of any bird, allowing it to glide effortlessly over oceans.'

In [ ]:
animals[1].metadata

{'type': 'bird',
 'number_of_legs': 2,
 'keywords': ['seabird', 'wingspan', 'ocean'],
 'habitat': 'marine',
 'tags': [{'a': 5, 'b': 8}, {'a': 8, 'b': 10}]}

In [ ]:
import getpass
import os
if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

Enter API key for OpenAI: ··········


In [ ]:
from langchain_openai import OpenAIEmbeddings

In [ ]:
embeddings=OpenAIEmbeddings(model="text-embedding-3-large")

In [ ]:
# embeddings.embed_query(animals[0].page_content)

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore

In [ ]:
vector_store=InMemoryVectorStore.from_documents(
    documents=animals,
    embedding=embeddings
)

In [ ]:
from graph_retriever.strategies import Eager

In [ ]:
from langchain_graph_retriever import GraphRetriever

In [ ]:
traversal_retriever = GraphRetriever(
    store = vector_store,
    edges = [("habitat", "habitat"), ("origin", "origin")],
    strategy = Eager(k=5, start_k=1, max_depth=2),
)

In [ ]:
traversal_retriever

GraphRetriever(store=<langchain_core.vectorstores.in_memory.InMemoryVectorStore object at 0x78a74ef441d0>, edges=[('habitat', 'habitat'), ('origin', 'origin')], strategy=Eager(select_k=5, start_k=1, adjacent_k=10, max_traverse=None, max_depth=2, _query_embedding=[]), adapter=<langchain_graph_retriever.adapters.in_memory.InMemoryAdapter object at 0x78a74d1cda10>)

In [ ]:
results=traversal_retriever.invoke("what animal could be found near a anaconda?")

In [ ]:
len(results)

5

In [ ]:
results[0]

Document(id='iguana', metadata={'_depth': 0, '_similarity_score': np.float64(0.39788762643161923), 'type': 'reptile', 'number_of_legs': 4, 'keywords': ['large lizard', 'herbivore', 'basking'], 'habitat': 'forest'}, page_content='iguanas are large herbivorous lizards often found basking in trees and near water.')

In [ ]:
from langchain.chat_models import init_chat_model

In [ ]:
llm = init_chat_model("gpt-4o-mini", model_provider="openai")

In [ ]:
llm.invoke("hi")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 8, 'total_tokens': 18, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b376dfbbd5', 'id': 'chatcmpl-BIzxHqVDsKinZDh7INj7MEbRvEa7x', 'finish_reason': 'stop', 'logprobs': None}, id='run-80add007-c89b-4f06-a431-38d8de2f08fe-0', usage_metadata={'input_tokens': 8, 'output_tokens': 10, 'total_tokens': 18, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

prompt = ChatPromptTemplate.from_template(
"""Answer the question based only on the context provided.

Context: {context}

Question: {question}"""
)

def format_docs(docs):
    return "\n\n".join(f"text: {doc.page_content} metadata: {doc.metadata}" for doc in docs)

In [ ]:
chain = (
    {"context": traversal_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
chain.invoke("what animal could be found near a anaconda?")

'Based on the context provided, iguanas, which are large herbivorous lizards often found near water, could be found near an anaconda.'

In [ ]:
chain.invoke("what animal could be found near a tiger?")

"The provided context mentions lions, gazelles, cheetahs, hyenas, and aardvarks but does not mention tigers. However, based on common knowledge, animals that could be found near a tiger often include prey animals like deer or livestock. Since no such animals are listed in the context, it's not possible to provide a specific answer from the information given. If considering animals within the text, lions (as large carnivorous cats) might be creatures that could be found in the same habitats, but they are not specifically found near tigers."

In [ ]:
chain.invoke("what animal can be found in south America?")

'Capybaras can be found in South America.'

In [ ]:
chain.invoke("what all animal can be found in south America give me all the names?")

'The animals mentioned in the context that can be found in South America are capybaras and herons.'